In [3]:
import os
import json
import pandas as pd
import torch

from src.model_handler import ModelHandler
from src.utils import test
import numpy as np

In [16]:
json_dir_path = "./exp_config_PCGNN_grid"
save_dir_path = "./experimental_results(gain)"
saved_model_path = os.path.join(save_dir_path, "saved_models")

train_ratio_l = [0.01, 0.4]
dataset_l = ["amazon_new", "yelp"]
for train_ratio in train_ratio_l:
        for dataset in dataset_l:
                df_path = os.path.join(save_dir_path, f"test_df/PCGNN-{dataset}.pkl")
                
                df = pd.read_pickle(df_path)
                
                mask = (df["train_ratio"] == train_ratio) | (df["train_ratio"] == str(train_ratio))
                sup_df = df[mask]
                semi_df = df[~mask]
                
                sup_ckpt_path_l = [os.path.join(saved_model_path, f"{exp_id}.pickle") for exp_id in sup_df["exp_id"]]
                sup_config_path_l = [os.path.join(json_dir_path, f"{exp_num}.json") for exp_num in sup_df["exp_num"]]
                
                semi_ckpt_path_l = [os.path.join(saved_model_path, f"{exp_id}.pickle") for exp_id in semi_df["exp_id"]]
                semi_config_path_l = [os.path.join(json_dir_path, f"{exp_num}.json") for exp_num in semi_df["exp_num"]]

                threshold_l = []
                auc_l = []
                f1_mac_l = []
                for ckpt_path, config_path in zip(sup_ckpt_path_l, sup_config_path_l):
                        with open(config_path, 'r') as f:
                                args = json.load(f)

                        handler = ModelHandler(args)
                        handler.model_select()
                        
                        model = handler.model.cuda()
                        model.load_state_dict(torch.load(ckpt_path))
                        
                        idx_valid, y_valid, idx_test, y_test = handler.dataset["idx_valid"], handler.dataset["y_valid"], handler.dataset["idx_test"], handler.dataset["y_test"]
                        
                        _, _, _, _, threshold = test(idx_valid, y_valid, model, 1024, flag="valid")
                        
                        auc_test, recall_test, f1_mac_test, precision, _ = test(idx_test, y_test, model, 1024, flag="test", valid_thresh=threshold)
                        
                        
                        threshold_l.append(threshold)
                        auc_l.append(auc_test)
                        f1_mac_l.append(f1_mac_test)
                
                print(f"===== {model}-{dataset}-{train_ratio} =====")
                print(f"AUC-ROC: {np.mean(auc_l):.4f}±{np.std(auc_l):.4f}")
                print(f"F1-macro: {np.mean(f1_mac_l):.4f}±{np.std(f1_mac_l):.4f}")
                print(f"Thresholds: {threshold_l}")
                print()

Run on amazon_new, postive/total num: 4847.0/9840, train num 78,valid num 2557, test num 5192, test positive num 545.0
Feature dimension: 25
Model: PCGNN, emb_size: 64.
Run on amazon_new, postive/total num: 4847.0/9840, train num 78,valid num 2557, test num 5192, test positive num 545.0
Feature dimension: 25
Model: PCGNN, emb_size: 64.
Run on amazon_new, postive/total num: 4847.0/9840, train num 78,valid num 2557, test num 5192, test positive num 545.0
Feature dimension: 25
Model: PCGNN, emb_size: 64.
===== PCALayer(
  (inter1): InterAgg3(
    (features): Embedding(9840, 25)
    (intra_agg1): IntraAgg(
      (features): Embedding(9840, 25)
    )
    (intra_agg2): IntraAgg(
      (features): Embedding(9840, 25)
    )
    (intra_agg3): IntraAgg(
      (features): Embedding(9840, 25)
    )
    (label_clf): Linear(in_features=25, out_features=2, bias=True)
  )
  (xent): CrossEntropyLoss()
)-amazon_new-0.01 =====
AUC-ROC: 0.8998±0.0209
F1-macro: 0.8985±0.0078
Thresholds: [0.4752525252525252